Description on what is done in this notebook
To be filled later

In [ ]:
# For using cuda GPU backend
from theano.sandbox import cuda

In [ ]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [ ]:
# Initializing default parameters
path = "data/ml-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

Let us know explore the data and see .

In [ ]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

#### We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.

In [ ]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [ ]:
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])
ratings.movieId = ratings.movieId.apply(lambda x : movieid2idx[x])

In [ ]:
ratings.head()

In [ ]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

In [ ]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

In [ ]:
# Let us decide on number of latent factors we''ll use
n_factors = 50

In [ ]:
# Splitting training and validation set
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [ ]:
len(val)

## basic model : Dot product of user and movie embeddings

In [ ]:
user_in = Input(shape=(1,),dtype = 'int64', name = 'user_in')
movie_in = Input(shape = (1,), dtype = 'int64', name = 'movie_in')

In [ ]:
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [ ]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [22]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80192 samples, validate on 19812 samples
Epoch 1/3
80192/80192 [==============================] - 6s - loss: 2.1510 - val_loss: 2.5758

In [23]:
model.optimizer.lr=0.001

In [24]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80192 samples, validate on 19812 samples
Epoch 1/6
80192/80192 [==============================] - 6s - loss: 2.0712 - val_loss: 2.5891